# OpenDrift forcing

In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
from dateutil.parser import parse
from itertools import repeat
from opendrift.readers import reader_netCDF_CF_unstructured
from opendrift.models.oceandrift import OceanDrift
from salishsea_tools import viz_tools

%matplotlib inline
plt.rcParams['font.size'] = 12

***

### Required forcing for OceanDrift module

In [3]:
# OceanDrift require variables
OceanDrift.required_variables

['x_sea_water_velocity',
 'y_sea_water_velocity',
 'x_wind',
 'y_wind',
 'land_binary_mask']

***
### Define paths

In [4]:
# Define paths
paths = {
    'NEMO': '/results/SalishSea/hindcast.201812',
    'NEMO_cutoff': '/results2/SalishSea/hindcast.201812_annex',
    'HRDPS': '/results/forcing/atmospheric/GEM2.5/operational',
    'out': '/ocean/bmoorema/research/MEOPAR/analysis-ben/data/opendrift_forcing',
    'date_cutoff': '2016 Nov 21',
}
mask = xr.open_dataset('/data/bmoorema/MEOPAR/grid/mesh_mask201702.nc')

***

### Prefix constructor for SSC

In [5]:
def make_prefix(date, paths, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., /results/SalishSea/hindcast.201812/ddmmmyy/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    path = paths['NEMO']
    if 'date_cutoff' in paths and date >= parse(paths['date_cutoff']):
        path = paths['NEMO_cutoff']
    fn = '_'.join([f'SalishSea_1{res}', *repeat(date.strftime('%Y%m%d'), 2)])
    prefix = os.path.join(path, date.strftime('%d%b%y').lower(), fn)
    
    return prefix

***

### Begin forcing processing

In [6]:
# Date
date = datetime(2019, 1, 1)

### NEMO

In [7]:
# Open NEMO velocity results as a list of raw=[u,v]
prefix = make_prefix(date, paths)
NEMO_out = os.path.join(paths['out'], os.path.split(prefix)[-1] + '_opendrift.nc')
raw = []
for k, key in zip(['U', 'V'], ['vozocrtx', 'vomecrty']):
    with xr.open_dataset(prefix + f'_grid_{k}.nc') as data:
        raw.append(data[key][:, 0, ...].values)
        if k is 'U': time = data.time_counter

# Unstagger and rotate NEMO velocities
u, v = viz_tools.rotate_vel(*viz_tools.unstagger(*raw))

# Save NEMO velocities
xr.Dataset(
    {
        'longitude': ('flat', mask.nav_lon[1:, 1:].values.reshape(-1)),
        'latitude': ('flat', mask.nav_lat[1:, 1:].values.reshape(-1)),
        'meshmask': ('flat', mask.tmask[0, 0, 1:, 1:].values.reshape(-1), {'standard_name': 'land_binary_mask'}),
        'u': (['time_counter', 'flat'], u.reshape(time.size, -1), {'standard_name': 'x_sea_water_velocity'}),
        'v': (['time_counter', 'flat'], v.reshape(time.size, -1), {'standard_name': 'y_sea_water_velocity'}),
    },
    coords={'time_counter': time}
).to_netcdf(NEMO_out)

### HRDPS

In [8]:
# Open HRDPS results and save flattened to netCDF
datestr = date.strftime('ops_y%Ym%md%d.nc')
HRDPS_out = os.path.join(paths['out'], datestr)
with xr.open_dataset(os.path.join(paths['HRDPS'], datestr)) as data:
    time = data.time_counter
    time.attrs['standard_name'] = 'time'
    xr.Dataset(
        {
            'longitude': ('flat', data.nav_lon.values.reshape(-1)),
            'latitude': ('flat', data.nav_lat.values.reshape(-1)),
            'u_wind': (['time_counter', 'flat'], data.u_wind.values.reshape(time.size, -1), {'standard_name': 'x_wind'}),
            'v_wind': (['time_counter', 'flat'], data.v_wind.values.reshape(time.size, -1), {'standard_name': 'y_wind'}),
        },
        coords={'time_counter': time},
    ).to_netcdf(HRDPS_out)

***

### Reading forcing files

In [9]:
# Read forcing
reader_netCDF_CF_unstructured.Reader(NEMO_out)

Reader: /ocean/bmoorema/research/MEOPAR/analysis-ben/data/opendrift_forcing/SalishSea_1h_20190101_20190101_opendrift.nc
Projection: 
  +proj=latlong
Coverage: [degrees]
  xmin: -126.211052   xmax: 0.000000
  ymin: 0.000000   ymax: 50.992344
  Corners (lon, lat):
    (-126.21,  50.99)  (  0.00,  50.99)
    (-126.21,   0.00)  (  0.00,   0.00)
Vertical levels [m]: 
  Not specified
Available time range:
  start: 2019-01-01 00:30:00   end: 2019-01-01 23:30:00   step: 1:00:00
    24 times (0 missing)
Variables:
  land_binary_mask
  x_sea_water_velocity
  y_sea_water_velocity
  time

In [10]:
# Read forcing
reader_netCDF_CF_unstructured.Reader(HRDPS_out)

Reader: /ocean/bmoorema/research/MEOPAR/analysis-ben/data/opendrift_forcing/ops_y2019m01d01.nc
Projection: 
  +proj=latlong
Coverage: [degrees]
  xmin: 230.833052   xmax: 240.529613
  ymin: 45.664901   ymax: 52.179264
  Corners (lon, lat):
    (230.83,  52.18)  (240.53,  52.18)
    (230.83,  45.66)  (240.53,  45.66)
Vertical levels [m]: 
  Not specified
Available time range:
  start: 2019-01-01 00:00:00   end: 2019-01-01 23:00:00   step: 1:00:00
    24 times (0 missing)
Variables:
  x_wind
  y_wind
  time